In [17]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Attempt imports for econml + LightGBM
try:
    from econml.dml import CausalForestDML
    from lightgbm import LGBMRegressor
    has_econml = True
except ImportError:
    has_econml = False
    print("econml or lightgbm not installed. Please install them to use the CF approach.")

############################################################################
# 1) DATASET + DATA GENERATION
############################################################################

def draw_x_shape(img_size=32, row=10, col=10):
    """
    Draw a small 'X' in the image at (row,col).
    We'll set a 3x3 cross for simplicity.
    """
    img = np.zeros((img_size, img_size), dtype=np.float32)
    # clamp row,col to avoid out-of-bounds
    row = min(max(row,1), img_size-2)
    col = min(max(col,1), img_size-2)
    # center
    img[row,   col]   = 1.0
    # diagonals
    img[row-1, col-1] = 1.0
    img[row+1, col+1] = 1.0
    img[row-1, col+1] = 1.0
    img[row+1, col-1] = 1.0
    return img

class XShapeAlphaBeta(Dataset):
    """
    For each sample:
      - random row,col in [0..(img_size-1)]
      - alpha = row / img_size
      - beta  = col / img_size
      - W ~ Bernoulli(0.5)
      - Y = alpha + beta*W + noise
      - image drawn with an 'X' at (row,col)
    """
    def __init__(self, n=10000, img_size=32, seed=42):
        super().__init__()
        rng = np.random.default_rng(seed)
        self.img_size = img_size
        self.samples  = []
        for _ in range(n):
            row = rng.integers(0, img_size)
            col = rng.integers(0, img_size)
            alpha = row / img_size
            beta  = col / img_size
            w     = rng.binomial(1, 0.5)
            noise = rng.standard_normal()
            y     = alpha + beta*w + noise
            img   = draw_x_shape(img_size, row, col)
            self.samples.append((img, w, y, alpha, beta))

        # "true" ATE ~ average beta
        betas = [s[4] for s in self.samples]
        self.true_ate = np.mean(betas)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]  # (img, w, y, alpha, beta)

def collate_fn_synthetic(batch):
    """
    batch => list of (img, w, y, alpha, beta)
    We'll produce X_t => (batch,1,H,W), W_t => (batch,1), Y_t => (batch,1), alpha,beta => 1D
    """
    imgs, w_list, y_list, alpha_list, beta_list = [], [], [], [], []
    for (img, w, y, a, b) in batch:
        imgs.append(img)
        w_list.append(w)
        y_list.append(y)
        alpha_list.append(a)
        beta_list.append(b)
    # shape => (batch, H, W)
    X_t = torch.tensor(imgs, dtype=torch.float32)
    X_t = X_t.unsqueeze(1)  # => (batch,1,H,W)
    W_t = torch.tensor(w_list, dtype=torch.float32).view(-1,1)
    Y_t = torch.tensor(y_list, dtype=torch.float32).view(-1,1)
    A_t = torch.tensor(alpha_list, dtype=torch.float32)
    B_t = torch.tensor(beta_list, dtype=torch.float32)
    return X_t, W_t, Y_t, A_t, B_t

############################################################################
# 2) MODELS
############################################################################

class MLPAlphaBeta(nn.Module):
    """
    Flatten => MLP => (alpha,beta).
    """
    def __init__(self, img_size=32, hidden_dim=128):
        super().__init__()
        in_dim = img_size*img_size
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)
        )
    def forward(self, x):
        # x => (batch, 1, H, W)
        b,c,h,w = x.shape
        x_flat = x.view(b, -1)  # => (b, H*W)
        ab_pred= self.net(x_flat)
        return ab_pred

class CNNAlphaBeta(nn.Module):
    """
    Convolution => final FC => (alpha,beta).
    """
    def __init__(self, img_size=32):
        super().__init__()
        # small conv net
        self.conv = nn.Sequential(
            nn.Conv2d(1,8, 3, padding=1),  # =>(8,H,W)
            nn.ReLU(),
            nn.MaxPool2d(2),              # =>(8,H/2,W/2)
            nn.Conv2d(8,16,3, padding=1), # =>(16,H/2,W/2)
            nn.ReLU(),
            nn.MaxPool2d(2)               # =>(16,H/4,W/4)
        )
        # for 32 => after 2 pool => 8 => shape=16*8*8=1024
        self.fc = nn.Sequential(
            nn.Linear(16*(img_size//4)*(img_size//4), 128),
            nn.ReLU(),
            nn.Linear(128,2)
        )
    def forward(self, x):
        b,c,h,w= x.shape
        h = self.conv(x)                # => (b,16,H/4,W/4)
        h = h.view(b, -1)               # =>(b,16*(H/4)*(W/4))
        ab= self.fc(h)                  # =>(b,2)
        return ab

############################################################################
# 3) TRAIN + EVAL
############################################################################

def dr_ate(alpha_hat, beta_hat, w_arr, y_arr):
    mu0= alpha_hat
    mu1= alpha_hat + beta_hat
    e= 0.5
    IF= (mu1 + w_arr*(y_arr - mu1)/e) - (mu0 + (1-w_arr)*(y_arr - mu0)/(1-e))
    ate= IF.mean()
    se=  IF.std(ddof=1)/np.sqrt(len(IF))
    return ate, se

def rmse(true_vals, pred_vals):
    return np.sqrt(np.mean((true_vals - pred_vals)**2))

def r2_score(true_vals, pred_vals):
    true_vals= np.array(true_vals)
    pred_vals= np.array(pred_vals)
    sse= np.sum((true_vals - pred_vals)**2)
    sst= np.sum((true_vals - np.mean(true_vals))**2)
    return 1.0 - sse/sst if sst>1e-8 else 0.0

def evaluate_model(model, loader, device='cpu'):
    """
    Evaluate param. model => (ATE,SE, RMSE(y), R2(y), RMSE(a), R2(a), RMSE(b), R2(b))
    CF doesn't produce alpha or Y => we'll handle separately
    """
    alphaP, betaP= [], []
    alphaT, betaT= [], []
    w_arr, y_arr= [], []
    y_pred= []
    model.eval()
    with torch.no_grad():
        for (X_t,W_t,Y_t,A_t,B_t) in loader:
            X_t= X_t.to(device)
            ab= model(X_t)  # =>(batch,2)
            alpha_hat= ab[:,0]
            beta_hat = ab[:,1]
            alphaP.extend(alpha_hat.cpu().numpy().tolist())
            betaP.extend(beta_hat.cpu().numpy().tolist())

            alphaT.extend(A_t.numpy().tolist())
            betaT.extend(B_t.numpy().tolist())
            w_arr.extend(W_t.view(-1).numpy().tolist())
            y_arr.extend(Y_t.view(-1).numpy().tolist())

            y_hat= alpha_hat + beta_hat*W_t.to(device).view(-1)
            y_pred.extend(y_hat.cpu().numpy().tolist())
    # arrays
    alphaP= np.array(alphaP)
    betaP=  np.array(betaP)
    alphaT= np.array(alphaT)
    betaT=  np.array(betaT)
    w_arr=  np.array(w_arr)
    y_arr=  np.array(y_arr)
    y_pred= np.array(y_pred)

    # ATE, SE
    ate_est, ate_se= dr_ate(alphaP, betaP, w_arr, y_arr)

    # RMSE+R2(Y)
    rmse_y= rmse(y_arr, y_pred)
    r2y   = r2_score(y_arr, y_pred)

    # RMSE+R2(alpha)
    rmse_a= rmse(alphaT, alphaP)
    r2a   = r2_score(alphaT, alphaP)

    # RMSE+R2(beta)
    rmse_b= rmse(betaT, betaP)
    r2b   = r2_score(betaT, betaP)
    return (ate_est, ate_se, rmse_y, r2y, rmse_a, r2a, rmse_b, r2b)

def train(model, loader, optimizer, epochs=15, device='cpu'):
    """Train param. model => MLP or CNN => predict (alpha,beta)."""
    mse= nn.MSELoss()
    model.train()
    for ep in tqdm(range(epochs), desc=f"{model.__class__.__name__} Train"):
        for (X_t,W_t,Y_t,_,_) in loader:
            X_t= X_t.to(device)
            W_t= W_t.to(device)
            Y_t= Y_t.to(device)
            ab= model(X_t)             # =>(batch,2)
            alpha_pred= ab[:,0:1]
            beta_pred= ab[:,1:2]
            y_hat= alpha_pred + beta_pred*W_t
            loss= mse(y_hat, Y_t)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

############################################################################
# 4) CAUSAL FOREST + UTIL
############################################################################

def prepare_cf_data(samples):
    """
    Flatten each (img) => X, also store W,Y,beta for CF metrics
    """
    X_list, W_list, Y_list, B_list= [],[],[],[]
    for (img,w,y,a,b) in samples:
        # flatten image => shape= (H*W,) => in this case 32*32=1024
        X_list.append(img.flatten())
        W_list.append(w)
        Y_list.append(y)
        B_list.append(b)
    X_np= np.array(X_list,dtype=np.float32)
    W_np= np.array(W_list,dtype=np.float32)
    Y_np= np.array(Y_list,dtype=np.float32)
    B_np= np.array(B_list,dtype=np.float32)
    return X_np, W_np, Y_np, B_np

############################################################################
# 5) MAIN
############################################################################

def main():
    device= 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device= {device}")

    # 1) We'll define a single test set
    test_ds= XShapeAlphaBeta(n=2000, img_size=32, seed=999)
    test_loader= DataLoader(test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn_synthetic)
    print(f"True ATE in test_ds= {test_ds.true_ate:.4f}\n")

    # 2) We'll run for these training set sizes
    train_sizes= [10_000]
    epochs= 500  # or more if you want

    # We'll store table rows as: (N, Model, ATE, SE, RMSEy, R2y, RMSEa, R2a, RMSEb, R2b)
    results= []

    def run_exp(N):
        # Build train dataset
        train_ds= XShapeAlphaBeta(n=N, img_size=32, seed=42)
        train_loader= DataLoader(train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn_synthetic)

        # (A) MLP
        mlp_model= MLPAlphaBeta(img_size=32).to(device)
        opt_mlp= optim.Adam(mlp_model.parameters(), lr=1e-3)
        train(mlp_model, train_loader, opt_mlp, epochs=epochs, device=device)
        mlp_res= evaluate_model(mlp_model, test_loader, device=device)

        # (B) CNN
        cnn_model= CNNAlphaBeta(img_size=32).to(device)
        opt_cnn= optim.Adam(cnn_model.parameters(), lr=1e-3)
        train(cnn_model, train_loader, opt_cnn, epochs=epochs, device=device)
        cnn_res= evaluate_model(cnn_model, test_loader, device=device)

        # (C) CF, if econml available
        if has_econml:
            from econml.dml import CausalForestDML
            from lightgbm import LGBMRegressor
            X_cf_train, W_cf_train, Y_cf_train, B_cf_train= prepare_cf_data(train_ds)
            X_cf_test,  W_cf_test,  Y_cf_test,  B_cf_test=  prepare_cf_data(test_ds)
            cf= CausalForestDML(
                model_y= LGBMRegressor(verbose=-1),
                model_t= LGBMRegressor(verbose=-1),
                n_estimators=400,
                min_samples_leaf=10,
                max_depth=25,
                random_state=42
            )
            cf.fit(Y_cf_train, W_cf_train, X=X_cf_train)
            b_hat_cf= cf.effect(X_cf_test)  # shape=(len(test),)
            ate_cf= np.mean(b_hat_cf)
            # naive se
            lb_cf, ub_cf= cf.effect_interval(X_cf_test)
            se_cf= (ub_cf - lb_cf).mean()/(2*1.96)

            # compare b_hat_cf to true beta => RMSE, R2
            def rmse(a,b): return np.sqrt(np.mean((a-b)**2))
            rb_cf= rmse(B_cf_test, b_hat_cf)
            r2b_cf= r2_score(B_cf_test, b_hat_cf)
            # CF => no alpha or Y => placeholders => (ate,se, None, None, None, None, rmseB, r2B)
            cf_res= (ate_cf, se_cf, None, None, None, None, rb_cf, r2b_cf)
        else:
            cf_res= (np.nan, np.nan, None, None, None, None, None, None)

        return mlp_res, cnn_res, cf_res

    # 3) Loop over train_sizes
    for N in train_sizes:
        mlpR, cnnR, cfR= run_exp(N)

        # MLP => (ate,se, rmse_y, r2_y, rmse_a, r2_a, rmse_b, r2_b)
        results.append( (N,"MLP", *mlpR) )
        # CNN => (ate,se, rmse_y, r2_y, rmse_a, r2_a, rmse_b, r2_b)
        results.append( (N,"CNN", *cnnR) )
        # CF => (ate,se, None, None, None, None, rb, r2b)
        results.append( (N,"CF",  *cfR) )

    # 4) Print final table
    print("\n=== RESULTS TABLE ===")
    print("DataN | Model |   ATE_est |   ATE_se |   RMSE(y) |  R2(y)  |  RMSE(a) |  R2(a)  |  RMSE(b) |  R2(b)")
    for row in results:
        (N,model,ate,se, ry,r2y, ra,r2a, rb,r2b)= row
        # if CF => no alpha,y
        if model=="CF":
            ate_s= f"{ate:8.4f}" if not np.isnan(ate) else "   --"
            se_s=  f"{se:8.4f}"  if not np.isnan(se)  else "   --"
            rb_s=  f"{rb:8.4f}"  if (rb is not None)  else "   --"
            r2b_s= f"{r2b:8.4f}" if (r2b is not None) else "   --"
            print(f"{N:<5} {model:<3} | {ate_s} | {se_s} |      --  |   --   |    --    |   --   | {rb_s} | {r2b_s}")
        else:
            ate_s= f"{ate:8.4f}"
            se_s=  f"{se:8.4f}"
            ry_s=  f"{ry:8.4f}"  if ry is not None else "   --"
            r2y_s= f"{r2y:8.4f}" if r2y is not None else "   --"
            ra_s=  f"{ra:8.4f}"  if ra is not None else "   --"
            r2a_s= f"{r2a:8.4f}" if r2a is not None else "   --"
            rb_s=  f"{rb:8.4f}"  if rb is not None else "   --"
            r2b_s= f"{r2b:8.4f}" if r2b is not None else "   --"
            print(f"{N:<5} {model:<3} | {ate_s} | {se_s} | {ry_s} | {r2y_s} | {ra_s} | {r2a_s} | {rb_s} | {r2b_s}")

    print(f"\nTrue ATE(test) = {test_ds.true_ate:.4f}")

if __name__=="__main__":
    main()


Using device= cpu
True ATE in test_ds= 0.4834



CNNAlphaBeta Train: 100%|██████████| 500/500 [17:00<00:00,  2.04s/it]



=== RESULTS TABLE ===
DataN | Model |   ATE_est |   ATE_se |   RMSE(y) |  R2(y)  |  RMSE(a) |  R2(a)  |  RMSE(b) |  R2(b)
10000 MLP |   0.4299 |   0.0478 |   1.1034 |  -0.0175 |   0.4583 |  -1.6184 |   0.6377 |  -3.8884
10000 CNN |   0.4292 |   0.0479 |   1.1038 |  -0.0183 |   0.4422 |  -1.4384 |   0.6228 |  -3.6626
10000 CF  |   0.4907 |   0.0207 |      --  |   --   |    --    |   --   |   0.2885 |  -0.0006

True ATE(test) = 0.4834
